# Verify ArUco Package Integration

This notebook verifies the `snap_fit.aruco` and `snap_fit.puzzle.sheet_aruco` modules.


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from pathlib import Path
import sys

import cv2
import matplotlib.pyplot as plt
import numpy as np

from snap_fit.aruco.aruco_board import ArucoBoardGenerator
from snap_fit.aruco.aruco_detector import ArucoDetector
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.puzzle.sheet_aruco import SheetAruco

## Test create and detect ArUco board


In [ ]:
# 1. Generate Board
board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
generator = ArucoBoardGenerator(board_config)
board_image = generator.generate_image()

plt.figure(figsize=(5, 7))
plt.imshow(board_image, cmap="gray")
plt.title("Generated Board")
plt.axis("off")
plt.show()

In [ ]:
# 2. Create Distorted Image
rows, cols = board_image.shape
src_points = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])  # pyright: ignore[reportArgumentType]
squeeze_amount = cols * 0.2
dst_points = np.float32(
    [
        [squeeze_amount, rows * 0.1],
        [cols - squeeze_amount, rows * 0.1],
        [0, rows],
        [cols, rows],
    ]  # pyright: ignore[reportArgumentType]
)
M = cv2.getPerspectiveTransform(src_points, dst_points)  # pyright: ignore[reportArgumentType, reportCallIssue]
distorted_image = cv2.warpPerspective(  # pyright: ignore[reportCallIssue]
    board_image,
    M,
    (cols, rows),
    borderValue=255,  # pyright: ignore[reportArgumentType]
)

distorted_path = Path("distorted_test.png")
cv2.imwrite(str(distorted_path), distorted_image)

plt.figure(figsize=(5, 7))
plt.imshow(distorted_image, cmap="gray")
plt.title("Distorted Image")
plt.axis("off")
plt.show()

In [ ]:
# 3. Test ArucoDetector directly
detector_config = ArucoDetectorConfig()
detector = ArucoDetector(generator, detector_config)
corners, ids, _ = detector.detect_markers(distorted_image)
rectified = detector.correct_perspective(distorted_image, corners, ids)  # pyright: ignore[reportArgumentType]

if rectified is not None:
    plt.figure(figsize=(5, 7))
    plt.imshow(rectified, cmap="gray")
    plt.title("Rectified Image (Direct)")
    plt.axis("off")
    plt.show()
else:
    print("Rectification failed")

In [ ]:
# 4. Test SheetAruco

# for reference

# class ArucoBoardConfig(BaseModelKwargs):
#     """Configuration for Aruco board generation."""
#     markers_x: int = Field(5, description="Number of markers in X direction")
#     markers_y: int = Field(7, description="Number of markers in Y direction")
#     marker_length: int = Field(100, description="Length of the marker side in pixels")
#     marker_separation: int = Field(
#         100, description="Separation between markers in pixels"
#     )
#     dictionary_id: int = Field(
#         cv2.aruco.DICT_6X6_250, description="ArUco dictionary ID"
#     )
#     margin: int = Field(20, description="Margin around the board in pixels")
#     border_bits: int = Field(1, description="Number of bits for the marker border")


# compute margin from configs of detector and board
crop_margin = (
    board_config.marker_length + board_config.margin + detector_config.rect_margin
)
lg.debug(f"Computed crop margin: {crop_margin}")

sheet_aruco = SheetAruco(detector, crop_margin=crop_margin)
sheet = sheet_aruco.load_sheet(distorted_path)

plt.figure(figsize=(5, 7))
plt.imshow(sheet.img_orig, cmap="gray")
plt.title("SheetAruco Loaded Image")
plt.axis("off")
plt.show()

## Test on actual data


In [ ]:
from snap_fit.params.snap_fit_params import get_snap_fit_paths

In [ ]:
paths = get_snap_fit_paths()
img_fol = paths.data_fol / "oca"
img_idx = 0
img_fp = list(img_fol.iterdir())[img_idx]
lg.debug(f"Testing on image: {img_fp}")

In [ ]:
sheet = sheet_aruco.load_sheet(img_fp)

plt.figure(figsize=(5, 7))
plt.imshow(sheet.img_orig, cmap="gray")
plt.title("SheetAruco Loaded Image")
plt.axis("off")
plt.show()